<a href="https://colab.research.google.com/github/jiyanshud22/Affective-Computing/blob/main/CS786A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS786** Assignment 3
---




### *Jiyanshu* *Dhaka* *220481*

# Q1. The paper uses inception v3 networks, but those are outdated. Can you reproduce the Study
# Design steps in the paper for inception v3 on inception v4 networks? [20 points]

# Q2. The paper studies whether CNNs can learn the concept of symmetry. Reproduce their results for global symmetry on synthetic data; you’re free to use their data and their code. [20 points]

# Q3. We discussed in class that there are a number of other gestalt principles beyond symmetry. Here is a fairly sensible list of five such principles. Test for one such principle – the principle of proximity. You basically need to figure out how to generate macro-objects that are made up of multiple microobjects and find a way for the CNN to classify the macro-objects without being confused by the presence of the micro-objects. For this question, I will grade on effort, not results.

# For example, a CNN trained to recognize the letter ‘I’ should interpret the display on the right in the figure above as three ‘I’s, generalizing across the empty space between the constituent circles (micro-objects) that make up each I (macro-object). I would design training sets that densely pack micro-objects to make macro-objects that are given class labels. I would also ensure that the microobjects don’t look anything like the macro-objects. Once the model is successfully detecting the macro-objects for this training set, I would take this trained model and retrain on training data with relatively looser packed micro-objects and repeat the whole things 3-4 rounds, increasing the gap between the micro-objects in each round until you get human-like proximity generalization behavior.[30 points]

# Q4. Test one other gestalt principle of your choice. For this one, you have to figure out the training and testing strategies for yourselves. [30 points]